In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
from sklearn.model_selection import KFold, train_test_split
import xgboost as xgb
import pickle as pkl
import importlib
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import ExtraTreesClassifier as ETC
from sklearn.ensemble import AdaBoostClassifier as ABC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.neural_network import MLPClassifier as MLPC
from xgboost.sklearn import XGBClassifier as XGBC
from sklearn import metrics
from functools import reduce
from sklearn.linear_model import LogisticRegression as LR
import scripts.xgboost_tuning as xgb_tuning
from sklearn.ensemble import BaggingClassifier as BC
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KernelDensity as KD
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 5, 5
import pyeeg
import pyrem.univariate as pruni
import scipy.signal as sig
import scipy.fftpack as fft


def recurrence_plot(series, epsilon=0.001):
    rp = (np.abs(series - series[:, np.newaxis]) < epsilon).astype(int)
    plt.imshow(rp, origin='lower', interpolation='none', cmap='Greys')
    plt.show()


def poincare_plot(series):
    plt.scatter(series[:-1], series[1:])
    plt.show()


def normalized_crosscorrelation(series1, series2):
    if series1.shape[0] != series2.shape[0]:
        raise BaseException('shapes must be the same')
    if (series1.shape[0] == 1) or (series2.shape[0] == 1):
        if np.isclose(series1[0], 0.0, rtol=0.0, atol=0.000001) or np.isclose(series2[0], 0.0, rtol=0.0,
                                                                              atol=0.000001):
            return 0.0
        if np.isclose(series1[0], series2[0], rtol=0.0, atol=0.000001):
            return 1.0
        if np.isclose(abs(series1[0]), abs(series2[0]), rtol=0.0, atol=0.000001):
            return -1.0
        return 0.0
    return ((series1 - series1.mean()) * (series2 - series2.mean())).sum() /\
            (series1.shape[0] * series1.std() * series2.std())


def normalized_autocorrelation(series, max_lag=76):
    result = []
    result.append(normalized_crosscorrelation(series, series))
    if max_lag <= 0:
        return np.array(result)
    if max_lag >= series.shape[0]:
        raise BaseException('max_lag must be not more than series length')
    temp_series = np.hstack((series, series))
    for k in range(1, max_lag + 1):
        result.append(normalized_crosscorrelation(temp_series[:-k], temp_series[k:]))
    return np.array(result)


def recurrence(series, epsilon=0.001):
    return (np.abs(series - series[:, np.newaxis]) < epsilon).sum() / (series.shape[0] * series.shape[0])


def mean_autocorrelation(series, max_lag=76):
    return normalized_autocorrelation(series, max_lag).mean()


def mean_period(series, max_lag=76, threshold=-1.0):
    '''left local minimum and all local maximums must be uniquely defined'''
    if series.shape[0] < 3:
        raise BaseException('time series must be longer')
    auto_corr = normalized_autocorrelation(series, max_lag)
    auto_corr[auto_corr < threshold] = threshold
    diffs = np.diff(auto_corr)
    prod_diffs = diffs[:-1] * diffs[1:]
    susp_extr_dots = np.where(prod_diffs < 0)[0]
    start_index = np.array([0])
    if (diffs[susp_extr_dots[0]] < 0) and (diffs[susp_extr_dots[0] + 1] > 0):
        if susp_extr_dots.shape[0] > 1:
            start_index = np.hstack((0, susp_extr_dots[1::2], series.shape[0]))
    else:
        start_index = np.hstack((0, susp_extr_dots[::2], series.shape[0]))
    if start_index.shape[0] == 1:
        start_index = np.hstack((start_index, series.shape[0]))
    return np.diff(start_index).mean()


def poincare_SD(series):
    pp = np.hstack((series[:-1][:, np.newaxis], series[1:][:, np.newaxis]))
    p1 = np.array([[-1.0 / np.sqrt(2.0)], [1.0 / np.sqrt(2.0)]])
    p2 = np.array([[1.0 / np.sqrt(2.0)], [1.0 / np.sqrt(2.0)]])
    return np.dot(pp, p1).ravel().std(), np.dot(pp, p2).ravel().std()


def DET_and_mean_diag_length(series, epsilon=0.001, min_l=15):
    if (min_l > series.shape[0]) or (min_l < 1):
        raise BaseException('min_l must be in correct range')
    rp = (np.abs(series - series[:, np.newaxis]) < epsilon).astype(np.int)
    lines_hist = np.zeros(series.shape[0]).astype(np.int)
    isline = False
    length = 0
    for j in range(1, series.shape[0]):
        for k in range(series.shape[0] - j):
            if rp[k][j + k]:
                if isline:
                    length += 1
                else:
                    isline = True
                    length = 1
            else:
                isline = False
                if length:
                    lines_hist[length - 1] += 1
                    length = 0
        isline = False
        if length:
            lines_hist[length - 1] += 1
            length = 0
    lines_hist *= 2
    lines_hist[-1] += 1
    line_lengths = np.arange(series.shape[0]) + 1
    mask = line_lengths >= min_l
    line_lengths[~mask] = 0
    sum_length = (line_lengths * lines_hist).sum()
    return sum_length / rp.sum(), sum_length / lines_hist[mask].sum()


def FilterBank(X, filters='BandpassBank', sample_rate=128, butter_pows=None):
    freq_pairs = None
    X_processed = None
    if filters == 'BandpassBank':
        freq_pairs =  [[0.5], [0.5, 4.0], [4.0, 8.0], [8.0, 13.0], [13.0, 30.0], [30.0, 42.0]]
        butter_pows = [5, 6, 8, 11, 11, 10]
    else:
        freq_pairs = filters
    for i in range(len(freq_pairs)):
        power = 5 if butter_pows is None else butter_pows[i]
        if len(freq_pairs[i]) == 1:
            b, a = sig.butter(power, 2 * freq_pairs[i][0] / sample_rate, btype='lowpass')
        else:
            b, a = sig.butter(power, 2 * np.array(freq_pairs[i]) / sample_rate, btype='bandpass')
        X_filtered = sig.lfilter(b, a, X, axis=0)
        X_processed = X_filtered if X_processed is None else np.c_[X_processed, X_filtered]
    return X_processed


def WindowDataset(dataset, interval=77, shift=1):
    if (interval < 1) or (interval > dataset.shape[0]):
        raise BaseException('interval has invalid value')
    if (shift < 1) or (shift > interval):
        raise BaseException('shift has invalid value')
    windowed_dataset = np.zeros((np.ceil((dataset.shape[0] - interval + 1) / shift).astype(np.int),
                                 interval, dataset.shape[1]))
    begin = 0
    for i in range(windowed_dataset.shape[0]):
        windowed_dataset[i] = dataset[begin: begin + interval, :].copy()
        begin += shift
    return windowed_dataset


def StackedRecurrence(X, epsilon=0.001):
    return np.sum(np.abs(X - X[:, np.newaxis]) < epsilon, axis=(0, 1)) / (X.shape[0] * X.shape[0])


def StackedDET(X, epsilon=0.001, min_l=15):
    result = np.zeros(2 * X.shape[1])
    for i in range(X.shape[1]):
        result[2 * i], result[2 * i + 1] = DET_and_mean_diag_length(series=X[:, i].ravel(),
                                                                    epsilon=epsilon, min_l=min_l)
    return result


def StackedPoincareSD(X):
    result = np.zeros(2 * X.shape[1])
    for i in range(X.shape[1]):
        result[2 * i], result[2 * i + 1] = poincare_SD(series=X[:, i].ravel())
    return result


def StackedMeanAutocorrelation(X, max_lag=76):
    result = np.zeros(X.shape[1])
    for i in range(X.shape[1]):
        result[i] = mean_autocorrelation(series=X[:, i].ravel(), max_lag=max_lag)
    return result


def StackedMeanPeriod(X, max_lag=76, threshold=-1.0):
    result = np.zeros(X.shape[1])
    for i in range(X.shape[1]):
        result[i] = mean_period(series=X[:, i].ravel(), max_lag=max_lag, threshold=threshold)
    return result


def StackedAproximateEntropy(X, M=None, R=None):
    if M is None:
        M = 14 + np.ones(X.shape[1]).astype(np.int)
    if R is None:
        R = 0.001 + np.zeros(X.shape[1])
    result = np.zeros(X.shape[1])
    for i in range(X.shape[1]):
        result[i] = pyeeg.ap_entropy(X=X[:, i].ravel(), M=M[i], R=R[i])
    return result


def StackedSampleEntropy(X, M=None, R=None):
    if M is None:
        M = 14 + np.ones(X.shape[1]).astype(np.int)
    if R is None:
        R = 0.001 + np.zeros(X.shape[1])
    result = np.zeros(X.shape[1])
    for i in range(X.shape[1]):
        result[i] = pyeeg.samp_entropy(X=X[:, i].ravel(), M=M[i], R=R[i])
    return result


def StackedSpectralEntropy(X, sampling_freq=128):
    result = np.zeros(X.shape[1])
    for i in range(X.shape[1]):
        result[i] = pruni.spectral_entropy(a=X[:, i].ravel(), sampling_freq=sampling_freq)
    return result


def StackedPFD(X):
    result = np.zeros(X.shape[1])
    for i in range(X.shape[1]):
        result[i] = pruni.pfd(a=X[:, i].ravel())
    return result


def StackedHjorth(X):
    result = np.zeros(3 * X.shape[1])
    for i in range(X.shape[1]):
        result[3 * i], result[3 * i + 1], result[3 * i + 2] = pruni.hjorth(X[:, i].ravel())
    return result


def TransformWindowedDataset(X_windowed, Y_windowed, parameters=None):
    if parameters is None:
        parameters = {'FilterBank': {'filters': [[0.5], [0.5, 4.0], [4.0, 8.0], [8.0, 13.0], [13.0, 30.0],
                                                 [30.0, 42.0]],
                                     'sample_rate': 128, 'butter_pows': [5, 6, 8, 11, 11, 10]},
                      'StackedRecurrence': {'epsilon': 0.001},
                      'StackedDET': {'epsilon': 0.001, 'min_l': 15},
                      'StackedPoincareSD': True,
                      'StackedMeanAutocorrelation': {'max_lag': 76},
                      'StackedMeanPeriod': {'max_lag': 76, 'threshold': -1.0},
                      'StackedAproximateEntropy': {'M': None, 'R': None},
                      'StackedSampleEntropy': {'M': None, 'R': None},
                      'StackedSpectralEntropy': {'sampling_freq': 128},
                      'StackedPFD': True,
                      'StackedHjorth': True,
                      'FFT': {'sampling_freq': 128},
                      'TargetWrite': 2}#4 -- begin of the window, 2 -- middle of the window, 1 -- end of the window
    features_num = X_windowed.shape[2]
    if parameters['FilterBank'] is not None:
        features_num *= len(parameters['FilterBank']['filters'])
    multiplier = 0
    if parameters['StackedRecurrence'] is not None:
        multiplier += 1
    if parameters['StackedDET'] is not None:
        multiplier += 2
    if parameters['StackedPoincareSD'] is not None:
        multiplier += 2
    if parameters['StackedMeanAutocorrelation'] is not None:
        multiplier += 1
    if parameters['StackedMeanPeriod'] is not None:
        multiplier += 1
    if parameters['StackedAproximateEntropy'] is not None:
        multiplier += 1
    if parameters['StackedSampleEntropy'] is not None:
        multiplier += 1
    if parameters['StackedSpectralEntropy'] is not None:
        multiplier += 1
    if parameters['StackedPFD'] is not None:
        multiplier += 1
    if parameters['StackedHjorth'] is not None:
        multiplier += 3
    features_num *= multiplier
    if parameters['FFT'] is not None:
        features_num += X_windowed.shape[2] * X_windowed.shape[1]
    if features_num == 0:
        raise BaseException('specify features')
    X_transformed = np.zeros((X_windowed.shape[0], features_num))
    Y_transformed = np.zeros((Y_windowed.shape[0], Y_windowed.shape[2])).astype(np.int)
    X_banded = None
    if parameters['FilterBank'] is not None:
        X_banded = np.zeros((X_windowed.shape[0], X_windowed.shape[1],
                             X_windowed.shape[2] * len(parameters['FilterBank']['filters'])))
        for i in range(X_windowed.shape[0]):
            X_banded[i] = FilterBank(X=X_windowed[i], filters=parameters['FilterBank']['filters'],
                                     sample_rate=parameters['FilterBank']['sample_rate'],
                                     butter_pows=parameters['FilterBank']['butter_pows'])
    else:
        X_banded = X_windowed
    print('    banded dataset')
    for i in range(X_banded.shape[0]):
        step = 0
        if parameters['StackedRecurrence'] is not None:
            X_transformed[i, step: step + X_banded.shape[2]] = StackedRecurrence(X=X_banded[i],
                                                               epsilon=parameters['StackedRecurrence']['epsilon'])
            step += X_banded.shape[2]
        if parameters['StackedDET'] is not None:
            X_transformed[i, step: step + 2 * X_banded.shape[2]] = StackedDET(X=X_banded[i],
                                                                   epsilon=parameters['StackedDET']['epsilon'],
                                                                   min_l=parameters['StackedDET']['min_l'])
            step += 2 * X_banded.shape[2]
        if parameters['StackedPoincareSD'] is not None:
            X_transformed[i, step: step + 2 * X_banded.shape[2]] = StackedPoincareSD(X=X_banded[i])
            step += 2 * X_banded.shape[2]
        if parameters['StackedMeanAutocorrelation'] is not None:
            X_transformed[i, step: step + X_banded.shape[2]] = StackedMeanAutocorrelation(X=X_banded[i],
                                                        max_lag=parameters['StackedMeanAutocorrelation']['max_lag'])
            step += X_banded.shape[2]
        if parameters['StackedMeanPeriod'] is not None:
            X_transformed[i, step: step + X_banded.shape[2]] = StackedMeanPeriod(X=X_banded[i],
                                                               max_lag=parameters['StackedMeanPeriod']['max_lag'],
                                                             threshold=parameters['StackedMeanPeriod']['threshold'])
            step += X_banded.shape[2]
        if parameters['StackedAproximateEntropy'] is not None:
            X_transformed[i, step: step + X_banded.shape[2]] = StackedAproximateEntropy(X=X_banded[i],
                                                               M=parameters['StackedAproximateEntropy']['M'],
                                                               R=parameters['StackedAproximateEntropy']['R'])
            step += X_banded.shape[2]
        if parameters['StackedSampleEntropy'] is not None:
            X_transformed[i, step: step + X_banded.shape[2]] = StackedSampleEntropy(X=X_banded[i],
                                                               M=parameters['StackedSampleEntropy']['M'],
                                                               R=parameters['StackedSampleEntropy']['R'])
            step += X_banded.shape[2]
        if parameters['StackedSpectralEntropy'] is not None:
            X_transformed[i, step: step + X_banded.shape[2]] = StackedSpectralEntropy(X=X_banded[i],
                                                sampling_freq=parameters['StackedSpectralEntropy']['sampling_freq'])
            step += X_banded.shape[2]
        if parameters['StackedPFD'] is not None:
            X_transformed[i, step: step + X_banded.shape[2]] = StackedPFD(X=X_banded[i])
            step += X_banded.shape[2]
        if parameters['StackedHjorth'] is not None:
            X_transformed[i, step: step + 3 * X_banded.shape[2]] = StackedHjorth(X=X_banded[i])
            step += 3 * X_banded.shape[2]
        if parameters['FFT'] is not None:
            X_transformed[i, step: step + X_windowed.shape[2] * X_windowed.shape[1]] = fft.rfft(
                                                                                    x=X_windowed[i], axis=0).ravel()
            step += X_windowed.shape[2] * X_windowed.shape[1]
        print('    step %i completed'%(i))
    print('    transformed design matrix')
    for i in range(Y_windowed.shape[0]):
        if parameters['TargetWrite'] == 4:
            Y_transformed[i] = Y_windowed[i, 0, :]
        elif parameters['TargetWrite'] == 1:
            Y_transformed[i] = Y_windowed[i, -1, :]
        else:
            Y_transformed[i] = Y_windowed[i, Y_windowed.shape[1] // 2, :]
    print('    transformed target matrix')
    return X_transformed, Y_transformed

In [2]:
dataset_np = pd.read_csv('./eeg_eye_state.csv', sep=',').as_matrix()
print('loaded dataset')
windowed_design_matrix = WindowDataset(dataset_np[:, :14], interval=77, shift=1)
windowed_target_matrix = WindowDataset(dataset_np[:, 14][:, np.newaxis], interval=77, shift=1)
print('windowed dataset')
train_design_windowed, test_design_windowed = windowed_design_matrix[:7452], windowed_design_matrix[7452:]
train_target_windowed, test_target_windowed = windowed_target_matrix[:7452], windowed_target_matrix[7452:]
print('completed train/test split')
scaler = StandardScaler()
scaler.fit(dataset_np[:7528, :14])
for i in range(train_design_windowed.shape[0]):
    train_design_windowed[i, :, :] = scaler.transform(train_design_windowed[i, :, :])
for i in range(test_design_windowed.shape[0]):
    test_design_windowed[i, :, :] = scaler.transform(test_design_windowed[i, :, :])
print('completed scaling dataset')
train_design_windowed, train_target_windowed = TransformWindowedDataset(train_design_windowed,
                                                                        train_target_windowed)
test_design_windowed, test_target_windowed = TransformWindowedDataset(test_design_windowed, test_target_windowed)
print('transformed dataset')
file = open('EEG_eye_state_train_X.pkl', 'wb')
pkl.dump(train_design_windowed, file)
file.close()
file = open('EEG_eye_state_train_Y.pkl', 'wb')
pkl.dump(train_target_windowed, file)
file.close()
file = open('EEG_eye_state_test_X.pkl', 'wb')
pkl.dump(test_design_windowed, file)
file.close()
file = open('EEG_eye_state_test_Y.pkl', 'wb')
pkl.dump(test_target_windowed, file)
file.close()
print('saved transformed dataset')

loaded dataset
windowed dataset
completed train/test split
completed scaling dataset
    banded dataset
    transformed design matrix
    transformed target matrix
    banded dataset
    transformed design matrix
    transformed target matrix
transformed dataset
saved transformed dataset


In [2]:
file = open('EEG_eye_state_train_X.pkl', 'rb')
train_X = pkl.load(file)
file.close()
file = open('EEG_eye_state_train_Y.pkl', 'rb')
train_Y = pkl.load(file).ravel()
file.close()
file = open('EEG_eye_state_test_X.pkl', 'rb')
test_X = pkl.load(file)
file.close()
file = open('EEG_eye_state_test_Y.pkl', 'rb')
test_Y = pkl.load(file).ravel()
file.close()
train_X.shape, test_X.shape, train_Y.shape, test_Y.shape

((7452, 2254), (7452, 2254), (7452,), (7452,))

In [3]:
from sklearn.decomposition import PCA


scaler = StandardScaler()
scaler.fit(train_X)
train_X, test_X = scaler.transform(train_X), scaler.transform(test_X)
transformer = PCA(n_components=0.95, svd_solver='full')
transformer.fit(train_X)
train_X, test_X = transformer.transform(train_X), transformer.transform(test_X)
scaler.fit(train_X)
train_X, test_X = scaler.transform(train_X), scaler.transform(test_X)
train_X.shape, test_X.shape, train_Y.shape, test_Y.shape

((7452, 532), (7452, 532), (7452,), (7452,))

In [4]:
from sklearn.svm import SVC


clf = SVC(probability=True)
clf.fit(train_X, train_Y)
preds = clf.predict_proba(test_X)[:, 1]
print('Support Vector Machine ROC AUC score:', roc_auc_score(test_Y, preds))
preds = clf.predict(test_X)
print('Support Vector Machine accuracy score:', accuracy_score(test_Y, preds))

Support Vector Machine ROC AUC score: 0.637569753399
Support Vector Machine accuracy score: 0.587761674718


In [5]:
clf = LR()
clf.fit(train_X, train_Y)
preds = clf.predict_proba(test_X)[:, 1]
print('Logistic Regression ROC AUC score:', roc_auc_score(test_Y, preds))
preds = clf.predict(test_X)
print('Logistic Regression accuracy score:', accuracy_score(test_Y, preds))

Logistic Regression ROC AUC score: 0.591191904632
Logistic Regression accuracy score: 0.550858829844


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [6]:
from sklearn.neighbors import KNeighborsClassifier as KNC


clf = KNC()
clf.fit(train_X, train_Y)
preds = clf.predict_proba(test_X)[:, 1]
print('K Nearest Neighbors ROC AUC score:', roc_auc_score(test_Y, preds))
preds = clf.predict(test_X)
print('K Nearest Neighbors accuracy score:', accuracy_score(test_Y, preds))

K Nearest Neighbors ROC AUC score: 0.569893762905
K Nearest Neighbors accuracy score: 0.555018786903


In [7]:
clf = DTC()
clf.fit(train_X, train_Y)
preds = clf.predict_proba(test_X)[:, 1]
print('Decision Tree ROC AUC score:', roc_auc_score(test_Y, preds))
preds = clf.predict(test_X)
print('Decision Tree accuracy score:', accuracy_score(test_Y, preds))

Decision Tree ROC AUC score: 0.556414196112
Decision Tree accuracy score: 0.540794417606


In [8]:
clf = RFC()
clf.fit(train_X, train_Y)
preds = clf.predict_proba(test_X)[:, 1]
print('Random Forest ROC AUC score:', roc_auc_score(test_Y, preds))
preds = clf.predict(test_X)
print('Random Forest accuracy score:', accuracy_score(test_Y, preds))

Random Forest ROC AUC score: 0.533742919475
Random Forest accuracy score: 0.541196994096


In [9]:
clf = ETC()
clf.fit(train_X, train_Y)
preds = clf.predict_proba(test_X)[:, 1]
print('Extra Trees ROC AUC score:', roc_auc_score(test_Y, preds))
preds = clf.predict(test_X)
print('Extra Trees ROC AUC score:', accuracy_score(test_Y, preds))

Extra Trees ROC AUC score: 0.54358779072
Extra Trees ROC AUC score: 0.539586688137


In [10]:
clf = ABC()
clf.fit(train_X, train_Y)
preds = clf.predict_proba(test_X)[:, 1]
print('AdaBoost ROC AUC score:', roc_auc_score(test_Y, preds))
preds = clf.predict(test_X)
print('AdaBoost ROC AUC score:', accuracy_score(test_Y, preds))

AdaBoost ROC AUC score: 0.563101328451
AdaBoost ROC AUC score: 0.533413848631


In [11]:
clf = XGBC()
clf.fit(train_X, train_Y)
preds = clf.predict_proba(test_X)[:, 1]
print('Extreme Gradient Boosting ROC AUC score:', roc_auc_score(test_Y, preds))
preds = clf.predict(test_X)
print('Extreme Gradient Boosting accuracy score:', accuracy_score(test_Y, preds))

Extreme Gradient Boosting ROC AUC score: 0.598028361058
Extreme Gradient Boosting accuracy score: 0.564143853999


In [12]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


clf = LDA()
clf.fit(train_X, train_Y)
preds = clf.predict_proba(test_X)[:, 1]
print('Linear Discriminant Analysis ROC AUC score:', roc_auc_score(test_Y, preds))
preds = clf.predict(test_X)
print('Linear Discriminant Analysis accuracy score:', accuracy_score(test_Y, preds))

Linear Discriminant Analysis ROC AUC score: 0.595051423897
Linear Discriminant Analysis accuracy score: 0.554079441761


/usr/local/lib/python3.5/dist-packages/sklearn/discriminant_analysis.py:523: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [13]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA


clf = QDA()
clf.fit(train_X, train_Y)
preds = clf.predict_proba(test_X)[:, 1]
print('Quadratic Discriminant Analysis ROC AUC score:', roc_auc_score(test_Y, preds))
preds = clf.predict(test_X)
print('Quadratic Discriminant Analysis accuracy score:', accuracy_score(test_Y, preds))

Quadratic Discriminant Analysis ROC AUC score: 0.511011453504
Quadratic Discriminant Analysis accuracy score: 0.386205045625


In [14]:
from sklearn.svm import SVR


clf = SVR()
clf.fit(train_X, train_Y)
preds = clf.predict(test_X)
print('Support Vector Machine ROC AUC score:', roc_auc_score(test_Y, preds))

Support Vector Machine ROC AUC score: 0.637829833651


In [15]:
from sklearn.neighbors import KNeighborsRegressor as KNR


clf = KNR()
clf.fit(train_X, train_Y)
preds = clf.predict(test_X)
print('K Nearest Neighbors ROC AUC score:', roc_auc_score(test_Y, preds))

K Nearest Neighbors ROC AUC score: 0.569893762905


In [16]:
from sklearn.tree import DecisionTreeRegressor as DTR


clf = DTR()
clf.fit(train_X, train_Y)
preds = clf.predict(test_X)
print('Decision Tree ROC AUC score:', roc_auc_score(test_Y, preds))

Decision Tree ROC AUC score: 0.555421247419


In [17]:
from sklearn.ensemble import RandomForestRegressor as RFR


clf = RFR()
clf.fit(train_X, train_Y)
preds = clf.predict(test_X)
print('Random Forest ROC AUC score:', roc_auc_score(test_Y, preds))

Random Forest ROC AUC score: 0.586031555573


In [18]:
from sklearn.ensemble import ExtraTreesRegressor as ETR


clf = ETR()
clf.fit(train_X, train_Y)
preds = clf.predict(test_X)
print('Extra Trees ROC AUC score:', roc_auc_score(test_Y, preds))

Extra Trees ROC AUC score: 0.594322256418


In [19]:
from sklearn.ensemble import AdaBoostRegressor as ABR


clf = ABR()
clf.fit(train_X, train_Y)
preds = clf.predict(test_X)
print('AdaBoost ROC AUC score:', roc_auc_score(test_Y, preds))

AdaBoost ROC AUC score: 0.582880166738


In [20]:
from xgboost.sklearn import XGBRegressor as XGBR


clf = XGBR()
clf.fit(train_X, train_Y)
preds = clf.predict(test_X)
print('Extreme Gradient Boosting ROC AUC score:', roc_auc_score(test_Y, preds))

Extreme Gradient Boosting ROC AUC score: 0.594125248354


In [21]:
dataset_np = pd.read_csv('./eeg_eye_state.csv', sep=',').as_matrix()

In [22]:
from sklearn.model_selection import cross_val_score
from mne.decoding import CSP
from sklearn.pipeline import make_pipeline


def CSPWindowDataset(dataset, interval=77, shift=1):
    if (interval < 1) or (interval > dataset.shape[0]):
        raise BaseException('interval has invalid value')
    if (shift < 1) or (shift > interval):
        raise BaseException('shift has invalid value')
    windowed_dataset = np.zeros((np.ceil((dataset.shape[0] - interval + 1) / shift).astype(np.int),
                                 dataset.shape[1], interval))
    begin = 0
    for i in range(windowed_dataset.shape[0]):
        windowed_dataset[i] = dataset[begin: begin + interval, :].copy().T
        begin += shift
    return windowed_dataset

In [23]:
b, a = sig.butter(5, 2 * np.array([1.0, 35]) / 128, btype='bandpass')
dataset_np[:, :14] = sig.lfilter(b, a, 1e-6*dataset_np[:, :14], axis=0)
windowed_dataset = CSPWindowDataset(dataset_np[:, :14])
windowed_target = WindowDataset(dataset_np[:, 14][:, np.newaxis])
target = np.zeros(windowed_target.shape[0]).astype(np.int)
for i in range(target.shape[0]):
    target[i] = windowed_target[i, windowed_target.shape[1] // 2, 0]

In [24]:
from sklearn.pipeline import Pipeline


csp = CSP(reg='ledoit_wolf')
cv = KFold(n_splits=3, shuffle=True, random_state=317)
algorithms = [(SVC(probability=True), 'Support Vector Machine'),
              (LR(), 'Logistic Regression'),
              (KNC(), 'k Nearest Neighbors'),
              (DTC(), 'Decision Tree'),
              (RFC(), 'Random Forest'),
              (ETC(), 'Extra Trees'),
              (ABC(), 'Ada Boost'),
              (XGBC(), 'Extreme Gradient Boosting'),
              (LDA(), 'Linear Discriminant Analysis'),
              (QDA(), 'Quadratic Discriminant Analysis'),
              (SVR(), 'Support Vector Machine (Regressor)'),
              (KNR(), 'k Nearest Neighbors (Regressor)'),
              (DTR(), 'Decision Tree (Regressor)'),
              (RFR(), 'Random Forest (Regressor)'),
              (ETR(), 'Extra Trees (Regressor)'),
              (ABR(), 'Ada Boost (Regressor)'),
              (XGBR(), 'Extreme Gradient Boosting (Regressor)')]
for alg, name in algorithms:
    clf = Pipeline([('CSP', csp), (name, alg)])
    scores = cross_val_score(clf, windowed_dataset, target, scoring='roc_auc', cv=cv.split(target), n_jobs=1)
    print('Pipeline cross validation completed:', name, 'algorithm')
    print('    mean roc auc is:', np.mean(scores))

Pipeline cross validation completed: Support Vector Machine algorithm
    mean roc auc is: 0.813186011431
Pipeline cross validation completed: Logistic Regression algorithm
    mean roc auc is: 0.546761021336
Pipeline cross validation completed: k Nearest Neighbors algorithm
    mean roc auc is: 0.990047623206
Pipeline cross validation completed: Decision Tree algorithm
    mean roc auc is: 0.923263249909
Pipeline cross validation completed: Random Forest algorithm
    mean roc auc is: 0.986248318377
Pipeline cross validation completed: Extra Trees algorithm
    mean roc auc is: 0.992163066001
Pipeline cross validation completed: Ada Boost algorithm
    mean roc auc is: 0.782153174349
Pipeline cross validation completed: Extreme Gradient Boosting algorithm
    mean roc auc is: 0.865977248196
Pipeline cross validation completed: Linear Discriminant Analysis algorithm
    mean roc auc is: 0.542754069951
Pipeline cross validation completed: Quadratic Discriminant Analysis algorithm
    me